In [11]:
import numpy as np
import random
from pathlib import Path
from LTTraceData import LTTraceData
from PyLTSpice import SimCommander, RawRead, SimRunner, SpiceEditor
import CoilcraftRandomSelect as crs
import MurataRandomSelect as mrs

high_side = True
low_side = False

In [13]:
def frequencytoltpulse(frequency, side: bool):
    deadband = 8  # ns
    period = 1 / (frequency / 1E9)  # Convert to Gigahertz to ensure the period will be in nanoseconds
    if side == low_side:
        return f"PULSE(3.3 0 0n 1n 1n {period + (deadband * 2)}n {period * 2}n)"
    else:
        return f"PULSE(0 3.3 {deadband}n 1n 1n {period}n {period * 2}n)"

def simulate_circuit(fsw: int, ind_index: int, cap1_index: int, cap2_index: int, cap3_index: int, cap4_index: int):
    ltc = SimRunner(output_folder='LTFiles')
    netlist = SpiceEditor('LTFiles/EPC23102_Mine.asc')

    netlist.set_component_value('VHSin', frequencytoltpulse(fsw, high_side))
    netlist.set_component_value('VLSin', frequencytoltpulse(fsw, low_side))

    netlist.add_instruction(crs.indexed_coilcraft_inductor(ind_index)['SubCkt'])
    netlist.add_instruction(mrs.indexed_murata_capacitor('C1', cap1_index)['SubCkt'])
    netlist.add_instruction(mrs.indexed_murata_capacitor('C2', cap2_index)['SubCkt'])
    netlist.add_instruction(mrs.indexed_murata_capacitor('C3', cap3_index)['SubCkt'])
    netlist.add_instruction(mrs.indexed_murata_capacitor('C4', cap4_index)['SubCkt'])

    # netlist.write_netlist(Path("LTFiles/FullyCustom.net"))
    # raw, log = ltc.run_now(netlist, run_filename="FullyCustom.net")
    raw, log = ltc.run_now(netlist, run_filename="Test")

    ltr = RawRead(raw)

    v_out = LTTraceData(ltr, 'V(vout)')
    print(v_out.min)
    print(v_out.max)
    print(v_out.average)
    v_out_max = v_out.max
    v_out_min = v_out.min

simulate_circuit(400000, 2906, 4000, 6000, 9000, 11500)

SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started
SimRunner started


OK
Tue May 16 16:36:46 2023 info: : Starting simulation 1
Tue May 16 16:38:18 2023 info: Simulation Successful. Time elapsed: 00:00:00
Tue May 16 16:38:18 2023 info: Simulation Finished. Calling...dummy_callback(rawfile, logfile)
Reading the file with encoding  utf_16_le
File contains 39 traces, reading 39
Binary RAW file with Normal access
-0.25221968
12.127667
4.0826626
